# 🐘 Introduction au Big Data & Systèmes Distribués

---

## 📋 Prérequis

| Niveau | Compétence |
|--------|------------|
| ✅ Requis | Avoir suivi le module `06_intro_relational_databases` |
| ✅ Requis | Avoir suivi le module `07_sql_for_data_engineers` |

## 🎯 Objectifs du module

À la fin de ce notebook, tu seras capable de :

- ✅ Comprendre ce qu'est le Big Data et ses caractéristiques (5V)
- ✅ Expliquer pourquoi le traitement distribué est nécessaire
- ✅ Décrire l'architecture Hadoop (HDFS, MapReduce, YARN)
- ✅ Comprendre le modèle MapReduce
- ✅ Expliquer pourquoi Spark a remplacé MapReduce
- ✅ **Différencier Data Lake et Data Lakehouse**
- ✅ **Comprendre les architectures Lambda et Kappa**
- ✅ **Connaître le concept de Data Mesh**
- ✅ **Comprendre l'architecture Medallion (Bronze/Silver/Gold)**
- ✅ Comprendre le NoSQL et ses différents types
- ✅ Savoir quand utiliser SQL vs NoSQL
- ✅ Connaître le théorème CAP

---

> 💡 **Note** : Ce module est théorique. La pratique viendra avec MongoDB (module suivant) et PySpark !

<a href="https://colab.research.google.com/github/diakite-data/data-engineering-bootcamp/blob/main/notebooks/beginner/08_intro_big_data_distributed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>> 💡 **Conseil** : Cliquez sur le badge ci-dessus pour exécuter ce notebook directement dans Google Colab (aucune installation requise).

---

## 🌍 1. C'est quoi le Big Data ?

### 📖 Définition

Le **Big Data** désigne des ensembles de données **tellement volumineux et complexes** qu'ils ne peuvent pas être traités par des outils traditionnels (Excel, bases SQL classiques, un seul serveur).

### 📅 Origine

Le terme a émergé dans les années **2000** avec l'explosion :

- 🌐 D'Internet et des réseaux sociaux
- 📱 Des smartphones
- 📡 Des capteurs IoT
- 💳 Des transactions en ligne

### 📊 Ordres de grandeur

```
1 Ko  (Kilooctet)   = 1 page de texte
1 Mo  (Mégaoctet)   = 1 photo HD
1 Go  (Gigaoctet)   = 1 film HD
1 To  (Téraoctet)   = 1 000 films HD
1 Po  (Pétaoctet)   = 1 000 To = 1 million de Go
1 Eo  (Exaoctet)    = 1 000 Po
1 Zo  (Zettaoctet)  = 1 000 Eo
```

### 🏢 Exemples concrets

| Entreprise | Volume de données |
|------------|-------------------|
| Facebook | ~4 **Po** générés par jour |
| Google | ~20 **Po** traités par jour |
| Netflix | ~60 **Po** de vidéos stockées |
| CERN (LHC) | ~1 **Po** par seconde pendant les expériences |

---

## 📊 2. Les 5V du Big Data

Les caractéristiques du Big Data sont souvent résumées par les **5V** :

```
                    ┌─────────────┐
                    │   VOLUME    │
                    │  (quantité) │
                    └──────┬──────┘
                           │
        ┌──────────────────┼──────────────────┐
        │                  │                  │
        ▼                  ▼                  ▼
┌───────────────┐  ┌───────────────┐  ┌───────────────┐
│   VELOCITY    │  │   VARIETY     │  │   VERACITY    │
│   (vitesse)   │  │  (diversité)  │  │  (fiabilité)  │
└───────────────┘  └───────────────┘  └───────────────┘
                           │
                           ▼
                    ┌─────────────┐
                    │    VALUE    │
                    │  (valeur)   │
                    └─────────────┘
```

---

### 📦 Volume — La quantité massive de données

| Défi | Solution |
|------|----------|
| Impossible de stocker sur un seul disque | Stockage distribué (HDFS, S3) |
| Impossible de charger en RAM | Traitement par partitions |

---

### ⚡ Velocity — La vitesse de génération et traitement

| Type | Exemple | Latence |
|------|---------|--------|
| **Batch** | Rapport mensuel | Heures |
| **Near real-time** | Dashboard | Minutes |
| **Real-time / Streaming** | Détection de fraude | Millisecondes |

---

### 🎨 Variety — La diversité des formats

> 💡 **Rappel** : Tu as déjà vu ça dans le module 06 !

| Type | Format | Exemple |
|------|--------|--------|
| **Structuré** | Tables, colonnes fixes | SQL, CSV |
| **Semi-structuré** | Schéma flexible | JSON, XML (MongoDB, Elasticsearch) |
| **Non-structuré** | Pas de schéma | Images, vidéos, texte libre |

---

### ✅ Veracity — La fiabilité des données

| Problème | Impact |
|----------|--------|
| Données manquantes | Résultats biaisés |
| Doublons | Comptages faux |
| Erreurs de saisie | Mauvaises décisions |
| Données obsolètes | Analyses non pertinentes |

> 💡 C'est là que le **Data Engineer** intervient : nettoyer, valider, transformer !

---

### 💎 Value — La valeur extraite

Les données n'ont de valeur que si on peut en tirer des **insights** :
- 📈 Prédictions (ML)
- 📊 Dashboards
- 🔔 Alertes
- 💰 Optimisation business

---

## ⚙️ 3. Pourquoi le traitement distribué ?

### 🚫 Limites d'une machine unique

Imaginons que tu dois traiter **10 To** de logs :

| Ressource | Limite typique | Problème |
|-----------|---------------|----------|
| **RAM** | 64-256 Go | 10 To ne tient pas en mémoire |
| **CPU** | 8-64 cœurs | Traitement séquentiel = trop lent |
| **Disque** | 500 Mo/s lecture | 10 To = 5+ heures juste pour lire |
| **Réseau** | Goulot d'étranglement | Transférer 10 To = des heures |

---

### 📈 Scale-Up vs Scale-Out

```
SCALE-UP (vertical)              SCALE-OUT (horizontal)
─────────────────────            ─────────────────────────

   ┌─────────────┐               ┌───┐ ┌───┐ ┌───┐ ┌───┐
   │             │               │   │ │   │ │   │ │   │
   │   MEGA      │               │ S │ │ S │ │ S │ │ S │
   │  SERVEUR    │      vs       │ 1 │ │ 2 │ │ 3 │ │ 4 │
   │   💪💪💪    │               │   │ │   │ │   │ │   │
   │             │               └───┘ └───┘ └───┘ └───┘
   └─────────────┘               
                                 Cluster de serveurs
   + Plus de RAM                 + Moins cher (commodity)
   + Plus de CPU                 + Scalabilité infinie
   - Très cher $$$               + Tolérance aux pannes
   - Limite physique             - Plus complexe
```

### ✅ Le Big Data utilise le Scale-Out !

Au lieu d'une machine surpuissante, on utilise un **cluster** de machines ordinaires.

---

## 🧠 4. Concepts clés du traitement distribué

### 🔄 Parallélisme vs Distribution

| Concept | Description | Exemple |
|---------|-------------|--------|
| **Parallélisme** | Plusieurs tâches en même temps sur **une machine** | Multi-threading |
| **Distribution** | Tâches réparties sur **plusieurs machines** | Cluster Hadoop/Spark |

---

### 📍 Data Locality — "Amener le code aux données"

```
❌ MAUVAIS : Déplacer les données vers le code
┌────────────────┐                ┌────────────────┐
│   Serveur 1    │  ══10 To══►    │   Serveur 2    │
│   (données)    │   réseau       │    (calcul)    │
└────────────────┘   lent !       └────────────────┘

✅ BON : Déplacer le code vers les données
┌────────────────┐                ┌────────────────┐
│   Serveur 1    │  ◄══code══     │   Serveur 2    │
│ données+calcul │   (petit)      │    (master)    │
└────────────────┘                └────────────────┘
```

> 💡 C'est le principe fondamental de Hadoop et Spark !

---

### 🛡️ Fault Tolerance — Tolérance aux pannes

Dans un cluster de 1000 machines, **des pannes arrivent tous les jours** !

| Stratégie | Description |
|-----------|-------------|
| **Réplication** | Copier les données sur plusieurs nœuds (HDFS : 3 copies) |
| **Checkpointing** | Sauvegarder l'état intermédiaire |
| **Lineage** | Recalculer les données perdues (Spark RDD) |

---

## 🐘 5. L'écosystème Hadoop

**Hadoop** est un framework open-source créé par Yahoo (2006), inspiré des papiers de Google (GFS, MapReduce).

### 🏗️ Architecture

```
┌─────────────────────────────────────────────────────────────┐
│                    ÉCOSYSTÈME HADOOP                        │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌─────────┐  ┌─────────┐  ┌─────────┐  ┌─────────┐       │
│   │  Hive   │  │   Pig   │  │  HBase  │  │  Sqoop  │  ...  │
│   │  (SQL)  │  │(scripts)│  │ (NoSQL) │  │ (import)│       │
│   └────┬────┘  └────┬────┘  └────┬────┘  └────┬────┘       │
│        │            │            │            │             │
│        └────────────┴─────┬──────┴────────────┘             │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │  MapReduce  │  ◄── Traitement         │
│                    │  (calcul)   │                         │
│                    └──────┬──────┘                         │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │    YARN     │  ◄── Ressources         │
│                    │ (scheduler) │                         │
│                    └──────┬──────┘                         │
│                           │                                 │
│                    ┌──────┴──────┐                         │
│                    │    HDFS     │  ◄── Stockage           │
│                    │  (fichiers) │                         │
│                    └─────────────┘                         │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

---

### 📁 HDFS — Hadoop Distributed File System

Système de fichiers distribué qui stocke les données sur plusieurs machines.

```
Fichier original : data.csv (300 Mo)
                      │
        ┌─────────────┼─────────────┐
        ▼             ▼             ▼
   ┌─────────┐   ┌─────────┐   ┌─────────┐
   │ Block 1 │   │ Block 2 │   │ Block 3 │   (128 Mo chacun)
   │ 128 Mo  │   │ 128 Mo  │   │  44 Mo  │
   └────┬────┘   └────┬────┘   └────┬────┘
        │             │             │
   Répliqué 3x   Répliqué 3x   Répliqué 3x
```

| Caractéristique | Valeur par défaut |
|-----------------|-------------------|
| Taille de bloc | 128 Mo |
| Facteur de réplication | 3 |
| Type d'accès | Write once, read many |

---

### 🎛️ YARN — Yet Another Resource Negotiator

Gestionnaire de ressources du cluster :

- Alloue CPU/RAM aux applications
- Gère la file d'attente des jobs
- Surveille l'état des nœuds

---

## 🔄 6. MapReduce — Le modèle de traitement

**MapReduce** est un modèle de programmation pour traiter de grandes quantités de données en parallèle.

### 📐 Les 3 étapes

| Étape | Action | Parallélisme |
|-------|--------|-------------|
| **Map** | Transformer chaque élément | ✅ Parallèle |
| **Shuffle** | Regrouper par clé | ⚠️ Réseau |
| **Reduce** | Agréger les valeurs | ✅ Parallèle |

---

### 📝 Exemple : Word Count

Compter les occurrences de chaque mot dans un texte.

```
ENTRÉE : "hello world hello"

═══════════════════════════════════════════════════════════════════
                              MAP
═══════════════════════════════════════════════════════════════════

  "hello world hello"   
         │
         ▼
  ┌──────────────────────────────────────┐
  │  (hello, 1)  (world, 1)  (hello, 1)  │
  └──────────────────────────────────────┘

═══════════════════════════════════════════════════════════════════
                            SHUFFLE
═══════════════════════════════════════════════════════════════════

  Regrouper par clé (mot) :
  
  hello → [1, 1]
  world → [1]

═══════════════════════════════════════════════════════════════════
                            REDUCE
═══════════════════════════════════════════════════════════════════

  hello → sum([1, 1]) → 2
  world → sum([1])    → 1

═══════════════════════════════════════════════════════════════════

SORTIE : { "hello": 2, "world": 1 }
```

---

### 🖥️ MapReduce sur un cluster

```
                         ┌─────────────┐
                         │   MASTER    │
                         │  (Driver)   │
                         └──────┬──────┘
                                │
           ┌────────────────────┼────────────────────┐
           │                    │                    │
           ▼                    ▼                    ▼
    ┌─────────────┐      ┌─────────────┐      ┌─────────────┐
    │   NODE 1    │      │   NODE 2    │      │   NODE 3    │
    ├─────────────┤      ├─────────────┤      ├─────────────┤
    │ Données:    │      │ Données:    │      │ Données:    │
    │ "hello"     │      │ "world"     │      │ "hello"     │
    ├─────────────┤      ├─────────────┤      ├─────────────┤
    │ MAP:        │      │ MAP:        │      │ MAP:        │
    │ (hello, 1)  │      │ (world, 1)  │      │ (hello, 1)  │
    └──────┬──────┘      └──────┬──────┘      └──────┬──────┘
           │                    │                    │
           └────────────────────┼────────────────────┘
                                │
                         SHUFFLE (réseau)
                                │
           ┌────────────────────┴────────────────────┐
           ▼                                         ▼
    ┌─────────────┐                           ┌─────────────┐
    │  REDUCER 1  │                           │  REDUCER 2  │
    ├─────────────┤                           ├─────────────┤
    │ hello: [1,1]│                           │ world: [1]  │
    │ → hello: 2  │                           │ → world: 1  │
    └─────────────┘                           └─────────────┘
```

> 💡 Chaque nœud traite **ses données localement** (data locality) !

---

## ⚠️ 7. Limites de MapReduce

### 😓 Problèmes de MapReduce

| Problème | Cause | Impact |
|----------|-------|--------|
| **Lent** | Écrit sur disque entre chaque étape | I/O intensif |
| **Verbose** | Code Java complexe | Productivité basse |
| **Batch only** | Pas de streaming | Pas de temps réel |
| **Pas de cache** | Relit les données à chaque job | Itérations lentes (ML) |

### 💾 Le problème du disque

```
MapReduce : DISQUE → Map → DISQUE → Shuffle → DISQUE → Reduce → DISQUE
                 ↑           ↑              ↑              ↑
                 └───────────┴──────────────┴──────────────┘
                              LENT ! (I/O disque)

Spark :     DISQUE → Map → MÉMOIRE → Shuffle → MÉMOIRE → Reduce
                            ↑                    ↑
                            └────────────────────┘
                              RAPIDE ! (in-memory)
```

---

### 🔥 Exemple : Algorithme itératif (ML)

Pour un algorithme qui fait 10 itérations sur les mêmes données :

| Framework | Comportement | Temps |
|-----------|-------------|-------|
| **MapReduce** | Relit les données du disque 10 fois | 😓 |
| **Spark** | Garde les données en mémoire, itère 10 fois | ⚡ |

---

## ⚡ 8. Spark — L'évolution

**Apache Spark** (2014) a été créé pour résoudre les limitations de MapReduce.

### 📊 Comparaison Hadoop MapReduce vs Spark

| Critère | Hadoop MapReduce | Apache Spark |
|---------|------------------|---------------|
| **Vitesse** | Lent (disque) | 100x plus rapide (mémoire) |
| **Facilité** | Java verbeux | Python, Scala, SQL |
| **Traitement** | Batch only | Batch + Streaming |
| **Itérations** | Lent (relit le disque) | Rapide (cache en RAM) |
| **Écosystème** | Hive, Pig, etc. | Spark SQL, MLlib, GraphX |
| **Stockage** | HDFS | HDFS, S3, Cassandra, etc. |

---

### 🏗️ Architecture Spark

```
┌─────────────────────────────────────────────────────────────┐
│                      APACHE SPARK                           │
├─────────────────────────────────────────────────────────────┤
│                                                             │
│   ┌───────────┐ ┌───────────┐ ┌───────────┐ ┌───────────┐  │
│   │ Spark SQL │ │ Streaming │ │   MLlib   │ │  GraphX   │  │
│   │  (SQL)    │ │ (temps    │ │   (ML)    │ │ (graphes) │  │
│   │           │ │   réel)   │ │           │ │           │  │
│   └─────┬─────┘ └─────┬─────┘ └─────┬─────┘ └─────┬─────┘  │
│         │             │             │             │        │
│         └─────────────┴──────┬──────┴─────────────┘        │
│                              │                              │
│                       ┌──────┴──────┐                      │
│                       │ Spark Core  │                      │
│                       │   (RDD)     │                      │
│                       └──────┬──────┘                      │
│                              │                              │
│         ┌────────────────────┼────────────────────┐        │
│         ▼                    ▼                    ▼        │
│   ┌──────────┐        ┌──────────┐        ┌──────────┐    │
│   │   YARN   │        │   Mesos  │        │Standalone│    │
│   └──────────┘        └──────────┘        └──────────┘    │
│                                                             │
└─────────────────────────────────────────────────────────────┘
```

> 💡 Spark peut fonctionner **sur YARN** (cluster Hadoop existant) ou en mode standalone.

---

### 🔮 Concepts Spark à venir 

| Concept | Description |
|---------|-------------|
| **RDD** | Resilient Distributed Dataset — collection distribuée |
| **DataFrame** | Comme un tableau avec colonnes (similaire à Pandas) |
| **Transformation** | Opération lazy (map, filter, groupBy) |
| **Action** | Déclenche le calcul (collect, count, show) |
| **Lazy Evaluation** | Rien ne s'exécute tant qu'une action n'est pas appelée |
| **Partition** | Morceau de données traité par un worker |

---

### 🎯 Ce que tu vas apprendre avec PySpark

- Créer et manipuler des DataFrames distribués
- Écrire des transformations SQL-like
- Lire/écrire des fichiers (CSV, Parquet, JSON)
- Optimiser les performances
- Construire des pipelines de données

---

## 🏗️ 9. Architectures de Stockage Modernes

Maintenant que tu connais le Big Data et le traitement distribué, voyons comment **organiser et stocker** ces données.

> 💡 **Rappel** : Tu as vu Data Warehouse et Data Mart dans le module 06. Ici, on complète avec Data Lake et Lakehouse !

---

### 🌊 9.1 Data Lake — Le lac de données

Un **Data Lake** est un système de stockage qui contient une **grande quantité de données brutes** dans leur format natif.

#### 📖 Définition

> "Un Data Lake est un référentiel centralisé qui permet de stocker toutes les données structurées et non structurées à n'importe quelle échelle."

#### 🎯 Caractéristiques

| Caractéristique | Description |
|-----------------|-------------|
| **Schema-on-read** | Pas de schéma défini à l'écriture, appliqué à la lecture |
| **Tous formats** | Structuré, semi-structuré, non-structuré |
| **Stockage brut** | Données dans leur format original |
| **Scalabilité** | Pétaoctets de données |
| **Coût faible** | Stockage object (S3, ADLS, GCS) |

#### 🏭 Exemples de Data Lakes

| Produit | Provider | Stockage |
|---------|----------|----------|
| **Amazon S3** | AWS | Object storage |
| **Azure Data Lake Storage (ADLS)** | Azure | Object storage |
| **Google Cloud Storage (GCS)** | GCP | Object storage |
| **HDFS** | On-premise | Distributed filesystem |

#### ⚠️ Le piège : Data Swamp (Marécage de données)

```
❌ DATA SWAMP — Ce qu'il faut éviter :

┌─────────────────────────────────────────────────────────────────┐
│                       DATA LAKE                                 │
│                                                                 │
│   📁 old_data/                  📁 test_final_v2_FINAL/         │
│   📁 backup_2019/               📁 john_analysis/               │
│   📄 data.csv                   📄 data_new.csv                 │
│   📄 data_copy.csv              📄 ???.parquet                  │
│   📁 temp/                      📁 DO_NOT_DELETE/               │
│                                                                 │
│   😱 Personne ne sait ce que contiennent ces fichiers !        │
│   😱 Pas de documentation                                       │
│   😱 Données dupliquées, obsolètes, invalides                   │
└─────────────────────────────────────────────────────────────────┘
```

> 💡 Sans **gouvernance**, un Data Lake devient un Data Swamp inutilisable !

---

### 🏠 9.2 Data Lakehouse — Le meilleur des deux mondes

Un **Data Lakehouse** combine les avantages du Data Lake (flexibilité, coût) et du Data Warehouse (performance, ACID).

#### 📖 Définition

> "Un Lakehouse est une nouvelle architecture qui combine les meilleurs éléments des Data Lakes et des Data Warehouses."

#### 🔑 Technologies clés

| Technologie | Description | Créateur |
|-------------|-------------|----------|
| **Delta Lake** | Format de table ACID sur Data Lake | Databricks |
| **Apache Iceberg** | Format de table open source | Netflix |
| **Apache Hudi** | Format de table pour upserts | Uber |

#### ✅ Ce que le Lakehouse apporte

```
┌─────────────────────────────────────────────────────────────────┐
│                      DATA LAKEHOUSE                             │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│   ✅ Transactions ACID (comme un Data Warehouse)               │
│   ✅ Schema enforcement & evolution                             │
│   ✅ Time Travel (historique des versions)                      │
│   ✅ Stockage sur object storage (comme un Data Lake)          │
│   ✅ Support batch ET streaming                                │
│   ✅ Format ouvert (Parquet + métadonnées)                     │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

---

### ⚖️ 9.3 Comparaison : Warehouse vs Lake vs Lakehouse

| Critère | Data Warehouse | Data Lake | Data Lakehouse |
|---------|----------------|-----------|----------------|
| **Données** | Structurées | Toutes | Toutes |
| **Schéma** | Schema-on-write | Schema-on-read | Schema-on-write |
| **ACID** | ✅ Oui | ❌ Non | ✅ Oui |
| **Coût** | 💰💰💰 Élevé | 💰 Faible | 💰 Faible |
| **Performance** | ⚡ Haute | ⚠️ Variable | ⚡ Haute |
| **Use cases** | BI, Reporting | ML, Data Science | BI + ML |
| **Time Travel** | ⚠️ Limité | ❌ Non | ✅ Oui |
| **Exemples** | Snowflake, Redshift | S3, ADLS | Delta Lake, Iceberg |

```
🕐 Évolution historique :

    1990s-2000s           2010s              2020s+
         │                  │                  │
         ▼                  ▼                  ▼
  ┌─────────────┐    ┌─────────────┐    ┌─────────────┐
  │    DATA     │    │    DATA     │    │    DATA     │
  │  WAREHOUSE  │ →  │    LAKE     │ →  │  LAKEHOUSE  │
  │             │    │             │    │             │
  │ Structuré   │    │ Tout format │    │ Best of     │
  │ Coûteux     │    │ Pas d'ACID  │    │ both worlds │
  └─────────────┘    └─────────────┘    └─────────────┘
```

> 🔮 **À venir** : Tu apprendras Delta Lake et Iceberg en détail dans le module intermédiaire !

---

### 🥉🥈🥇 9.4 Medallion Architecture — Bronze, Silver, Gold

La **Medallion Architecture** (ou architecture en médaillons) est un pattern d'organisation des données dans un Data Lakehouse. Elle structure les données en **3 couches** de qualité croissante.

> 💡 Cette architecture est très populaire avec **Delta Lake** et **Databricks**, mais s'applique à tout Lakehouse.

---

#### 🏗️ Vue d'ensemble

```
┌─────────────────────────────────────────────────────────────────────────┐
│                     MEDALLION ARCHITECTURE                              │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│   SOURCES              BRONZE            SILVER            GOLD         │
│   ───────              ──────            ──────            ────         │
│                                                                         │
│  ┌─────────┐        ┌─────────┐       ┌─────────┐       ┌─────────┐    │
│  │   API   │───────►│         │       │         │       │         │    │
│  └─────────┘        │  Raw    │       │ Cleaned │       │ Curated │    │
│  ┌─────────┐        │  Data   │──────►│  Data   │──────►│  Data   │    │
│  │   DB    │───────►│         │       │         │       │         │    │
│  └─────────┘        │  🥉     │       │  🥈     │       │  🥇     │    │
│  ┌─────────┐        │ BRONZE  │       │ SILVER  │       │  GOLD   │    │
│  │  Files  │───────►│         │       │         │       │         │    │
│  └─────────┘        └─────────┘       └─────────┘       └─────────┘    │
│  ┌─────────┐                                                  │        │
│  │ Streams │───────►        ...                               ▼        │
│  └─────────┘                                           ┌───────────┐   │
│                                                        │ Dashboard │   │
│                                                        │ ML Models │   │
│                                                        │ Reports   │   │
│                                                        └───────────┘   │
└─────────────────────────────────────────────────────────────────────────┘
```

---

#### 🥉 Couche Bronze — Données brutes

La couche **Bronze** contient les données **exactement comme elles arrivent** des sources.

| Caractéristique | Description |
|-----------------|-------------|
| **Contenu** | Données brutes, non transformées |
| **Format** | Copie exacte des sources (JSON, CSV, logs...) |
| **Qualité** | Aucun nettoyage, peut contenir des erreurs |
| **But** | Historique complet, traçabilité, replay |
| **Rétention** | Longue durée (mois/années) |

```
🥉 BRONZE — Exemple de données brutes

┌────────────────────────────────────────────────────────────────┐
│ {                                                              │
│   "event_id": "abc123",                                       │
│   "timestamp": "2024-01-15T14:30:00Z",                        │
│   "user_id": "usr_456",                                       │
│   "action": "purchase",                                       │
│   "amount": "99.99",      ← String au lieu de number         │
│   "product": null,         ← Valeur manquante                 │
│   "_ingested_at": "2024-01-15T14:30:05Z"  ← Métadonnée ajoutée│
│ }                                                              │
└────────────────────────────────────────────────────────────────┘
```

---

#### 🥈 Couche Silver — Données nettoyées

La couche **Silver** contient les données **nettoyées, validées et conformées**.

| Caractéristique | Description |
|-----------------|-------------|
| **Contenu** | Données nettoyées et validées |
| **Transformations** | Types corrigés, doublons supprimés, nulls gérés |
| **Qualité** | Données fiables, schéma appliqué |
| **But** | Source de vérité pour les analyses |
| **Structure** | Tables normalisées ou semi-normalisées |

```
🥈 SILVER — Données nettoyées

┌────────────────────────────────────────────────────────────────┐
│ Table: silver_events                                          │
│ ┌───────────┬────────────────────┬──────────┬─────────┬──────┐│
│ │ event_id  │     timestamp      │ user_id  │ action  │amount││
│ ├───────────┼────────────────────┼──────────┼─────────┼──────┤│
│ │ abc123    │ 2024-01-15 14:30:00│ usr_456  │purchase │ 99.99││
│ │ def789    │ 2024-01-15 14:31:00│ usr_789  │ view    │  0.00││
│ └───────────┴────────────────────┴──────────┴─────────┴──────┘│
│                                                                │
│ ✅ Types corrects (DECIMAL pour amount)                       │
│ ✅ Pas de doublons                                            │
│ ✅ Valeurs nulles remplacées par défauts                      │
└────────────────────────────────────────────────────────────────┘
```

---

#### 🥇 Couche Gold — Données agrégées pour le business

La couche **Gold** contient les données **agrégées et optimisées** pour des cas d'usage métier spécifiques.

| Caractéristique | Description |
|-----------------|-------------|
| **Contenu** | Agrégations, KPIs, métriques business |
| **Structure** | Tables dénormalisées (Star Schema) |
| **Qualité** | Données prêtes pour dashboards et ML |
| **But** | Consommation directe par les analystes |
| **Performance** | Optimisées pour les requêtes |

```
🥇 GOLD — Données agrégées

┌────────────────────────────────────────────────────────────────┐
│ Table: gold_daily_sales                                       │
│ ┌────────────┬─────────────┬───────────┬───────────┬─────────┐│
│ │    date    │   region    │ category  │total_sales│num_orders││
│ ├────────────┼─────────────┼───────────┼───────────┼─────────┤│
│ │ 2024-01-15 │   France    │Electronics│ 125,430.50│    1,245││
│ │ 2024-01-15 │   Germany   │Electronics│  98,200.00│      987││
│ │ 2024-01-15 │   France    │  Fashion  │  45,670.00│      567││
│ └────────────┴─────────────┴───────────┴───────────┴─────────┘│
│                                                                │
│ → Prêt pour Power BI, Tableau, Looker                         │
│ → Requêtes ultra-rapides                                      │
└────────────────────────────────────────────────────────────────┘
```

---

#### 📊 Résumé des 3 couches

| Couche | Qualité | Transformations | Utilisateurs | Format typique |
|--------|---------|-----------------|--------------|----------------|
| 🥉 **Bronze** | Brute | Aucune | Data Engineers | JSON, CSV, Parquet brut |
| 🥈 **Silver** | Nettoyée | Cleaning, validation, typing | Data Engineers, Analysts | Delta Lake, Iceberg |
| 🥇 **Gold** | Business-ready | Agrégations, dénormalisation | Analysts, Dashboard, ML | Tables Star Schema |

---

#### ✅ Avantages de Medallion Architecture

| Avantage | Description |
|----------|-------------|
| **Traçabilité** | On peut toujours revenir aux données brutes (Bronze) |
| **Qualité progressive** | Chaque couche améliore la qualité |
| **Réutilisabilité** | Silver sert de source commune à plusieurs tables Gold |
| **Debugging** | Facile de trouver où une erreur s'est introduite |
| **Performance** | Gold optimisé pour les requêtes analytiques |

---

#### 🔄 Flux de données typique

```
┌──────────────────────────────────────────────────────────────────────────┐
│                         PIPELINE ETL/ELT                                 │
├──────────────────────────────────────────────────────────────────────────┤
│                                                                          │
│   SOURCE          BRONZE              SILVER              GOLD           │
│      │               │                   │                  │            │
│      │   Ingestion   │    Cleaning       │   Aggregation    │            │
│      │   (raw copy)  │    Validation     │   Denormalization│            │
│      │               │    Deduplication  │   Business rules │            │
│      ▼               ▼                   ▼                  ▼            │
│  ┌───────┐      ┌────────┐         ┌──────────┐       ┌──────────┐      │
│  │  API  │─────►│ bronze_│────────►│ silver_  │──────►│ gold_    │      │
│  │ logs  │      │ events │         │ events   │   ┌──►│ daily_   │      │
│  └───────┘      └────────┘         └──────────┘   │   │ metrics  │      │
│                                         │        │   └──────────┘      │
│  ┌───────┐      ┌────────┐         ┌────▼─────┐   │   ┌──────────┐      │
│  │  DB   │─────►│ bronze_│────────►│ silver_  │───┼──►│ gold_    │      │
│  │ users │      │ users  │         │ users    │   │   │ user_    │      │
│  └───────┘      └────────┘         └──────────┘   │   │ segments │      │
│                                                   │   └──────────┘      │
│                                                   │   ┌──────────┐      │
│                                                   └──►│ gold_    │      │
│                                                       │ funnel   │      │
│                                                       └──────────┘      │
└──────────────────────────────────────────────────────────────────────────┘
```

> 🔮 **À venir** : Tu implémenteras cette architecture avec Delta Lake dans le module intermédiaire !

---

## 🔄 10. Architectures de Traitement — Lambda & Kappa

Comment organiser le traitement des données ? Deux grandes architectures dominent.

---

### ⚡ 10.1 Lambda Architecture — Batch + Streaming

L'architecture **Lambda** (Nathan Marz, 2011) combine traitement **batch** et **streaming** pour avoir le meilleur des deux.

#### 🏗️ Structure

```
                         ┌─────────────────────────────────────┐
                         │            DATA SOURCES             │
                         │     (Events, Logs, Transactions)    │
                         └─────────────────┬───────────────────┘
                                           │
                    ┌──────────────────────┴──────────────────────┐
                    │                                             │
                    ▼                                             ▼
         ┌─────────────────────┐                      ┌─────────────────────┐
         │    BATCH LAYER      │                      │    SPEED LAYER      │
         │   (Traitement lot)  │                      │   (Temps réel)      │
         │                     │                      │                     │
         │  • Hadoop/Spark     │                      │  • Kafka Streams    │
         │  • Données complètes│                      │  • Spark Streaming  │
         │  • Haute précision  │                      │  • Flink            │
         │  • Latence: heures  │                      │  • Latence: secondes│
         └──────────┬──────────┘                      └──────────┬──────────┘
                    │                                             │
                    │     ┌─────────────────────────┐             │
                    │     │     SERVING LAYER       │             │
                    └────►│  (Couche de service)    │◄────────────┘
                          │                         │
                          │  • Combine les deux     │
                          │  • Requêtes rapides     │
                          │  • Vue unifiée          │
                          └───────────┬─────────────┘
                                      │
                                      ▼
                          ┌─────────────────────────┐
                          │      APPLICATIONS       │
                          │   Dashboards, APIs, ML  │
                          └─────────────────────────┘
```

#### 📋 Les 3 couches

| Couche | Rôle | Caractéristiques |
|--------|------|------------------|
| **Batch Layer** | Traitement complet, précis | Latence haute, données historiques |
| **Speed Layer** | Traitement temps réel | Latence basse, données récentes |
| **Serving Layer** | Combine et sert les données | Requêtes rapides, vue unifiée |

#### ✅ Avantages / ❌ Inconvénients

| ✅ Avantages | ❌ Inconvénients |
|--------------|------------------|
| Robuste (double traitement) | Complexe (2 systèmes à maintenir) |
| Données historiques complètes | Code dupliqué (batch + streaming) |
| Faible latence possible | Coût élevé (2 pipelines) |

---

### 🌀 10.2 Kappa Architecture — Streaming-first

L'architecture **Kappa** (Jay Kreps, 2014) simplifie Lambda en utilisant **uniquement du streaming**.

#### 🏗️ Structure

```
                         ┌─────────────────────────────────────┐
                         │            DATA SOURCES             │
                         │     (Events, Logs, Transactions)    │
                         └─────────────────┬───────────────────┘
                                           │
                                           ▼
                         ┌─────────────────────────────────────┐
                         │       MESSAGE QUEUE / LOG           │
                         │           (Apache Kafka)            │
                         │                                     │
                         │   • Stocke TOUS les événements      │
                         │   • Rétention longue (jours/mois)   │
                         │   • Replay possible                 │
                         └─────────────────┬───────────────────┘
                                           │
                                           ▼
                         ┌─────────────────────────────────────┐
                         │        STREAM PROCESSING            │
                         │    (Kafka Streams, Flink, Spark)    │
                         │                                     │
                         │   • Une seule codebase              │
                         │   • Traite events en temps réel     │
                         │   • Reprocessing = replay du log    │
                         └─────────────────┬───────────────────┘
                                           │
                                           ▼
                         ┌─────────────────────────────────────┐
                         │         SERVING LAYER               │
                         │   (Base de données, Cache, API)     │
                         └─────────────────────────────────────┘
```

#### 🔄 Le concept clé : Replay

```
Besoin de recalculer les données ?

Lambda : Relance le batch job (heures)
Kappa  : Rejoue le log Kafka (même vitesse que le streaming)

┌──────────────────────────────────────────────────────────────┐
│  Kafka Log (rétention 30 jours)                              │
│  ┌────┬────┬────┬────┬────┬────┬────┬────┬────┬────┐        │
│  │ E1 │ E2 │ E3 │ E4 │ E5 │ E6 │ E7 │ E8 │ E9 │E10 │ ...   │
│  └────┴────┴────┴────┴────┴────┴────┴────┴────┴────┘        │
│                              ▲                               │
│                              │                               │
│                    Consumer peut "remonter"                  │
│                    et rejouer les événements                 │
└──────────────────────────────────────────────────────────────┘
```

#### ✅ Avantages / ❌ Inconvénients

| ✅ Avantages | ❌ Inconvénients |
|--------------|------------------|
| Simple (1 seul système) | Stockage Kafka coûteux (rétention longue) |
| Une seule codebase | Pas idéal pour analytics très complexes |
| Reprocessing facile | Nécessite Kafka bien configuré |

---

### ⚖️ 10.3 Lambda vs Kappa — Quand utiliser quoi ?

| Critère | Lambda | Kappa |
|---------|--------|-------|
| **Complexité** | Haute (2 systèmes) | Moyenne (1 système) |
| **Maintenance** | 2 codebases | 1 codebase |
| **Reprocessing** | Batch job (lent mais fiable) | Replay Kafka (rapide) |
| **Use case idéal** | Analytics historiques + temps réel | Event-driven, streaming-first |
| **Coût** | Élevé | Moyen |

#### 🎯 Recommandations

| Situation | Architecture recommandée |
|-----------|-------------------------|
| Besoin de rapports historiques précis + temps réel | **Lambda** |
| Application principalement event-driven | **Kappa** |
| Équipe petite, budget limité | **Kappa** |
| Données complexes nécessitant du batch lourd | **Lambda** |
| Startup, MVP, itération rapide | **Kappa** |

> 💡 **En pratique** : Beaucoup d'entreprises utilisent des **architectures hybrides** adaptées à leurs besoins spécifiques.

---

## 🕸️ 11. Data Mesh — L'approche décentralisée

Le **Data Mesh** est une approche **organisationnelle** (pas technique) pour gérer les données à grande échelle.

### 📖 Origine du problème

```
Architecture centralisée traditionnelle :

    Domain A        Domain B        Domain C
        │               │               │
        └───────────────┼───────────────┘
                        │
                        ▼
              ┌─────────────────────┐
              │   DATA TEAM         │
              │   CENTRALISÉE       │  ← Goulot d'étranglement !
              │                     │
              │ • Pipeline A        │
              │ • Pipeline B        │
              │ • Pipeline C        │
              └─────────────────────┘
                        │
                        ▼
              ┌─────────────────────┐
              │   DATA WAREHOUSE    │
              └─────────────────────┘

❌ Problèmes :
  • L'équipe data devient un goulot d'étranglement
  • Elle ne comprend pas tous les domaines métiers
  • Temps de livraison très long
```

---

### 🎯 Les 4 principes du Data Mesh

| Principe | Description |
|----------|-------------|
| **1. Domain Ownership** | Chaque domaine métier est responsable de ses données |
| **2. Data as a Product** | Les données sont traitées comme des produits avec SLA, qualité, documentation |
| **3. Self-serve Platform** | Plateforme en libre-service pour créer des data products |
| **4. Federated Governance** | Gouvernance fédérée avec des standards communs |

---

### 🏗️ Architecture Data Mesh

```
┌─────────────────────────────────────────────────────────────────────────┐
│                         DATA MESH                                       │
├─────────────────────────────────────────────────────────────────────────┤
│                                                                         │
│  ┌───────────────────┐  ┌───────────────────┐  ┌───────────────────┐   │
│  │   DOMAIN: Sales   │  │  DOMAIN: Product  │  │ DOMAIN: Customer  │   │
│  │                   │  │                   │  │                   │   │
│  │  ┌─────────────┐  │  │  ┌─────────────┐  │  │  ┌─────────────┐  │   │
│  │  │ Data Product│  │  │  │ Data Product│  │  │  │ Data Product│  │   │
│  │  │ "Orders"    │  │  │  │ "Catalog"   │  │  │  │ "Profiles"  │  │   │
│  │  └─────────────┘  │  │  └─────────────┘  │  │  └─────────────┘  │   │
│  │                   │  │                   │  │                   │   │
│  │  Owner: Sales Team│  │  Owner: Product   │  │  Owner: CRM Team  │   │
│  └─────────┬─────────┘  └─────────┬─────────┘  └─────────┬─────────┘   │
│            │                      │                      │             │
│            └──────────────────────┼──────────────────────┘             │
│                                   │                                     │
│                    ┌──────────────▼──────────────┐                     │
│                    │    SELF-SERVE PLATFORM      │                     │
│                    │  (Infrastructure commune)   │                     │
│                    └──────────────┬──────────────┘                     │
│                                   │                                     │
│                    ┌──────────────▼──────────────┐                     │
│                    │   FEDERATED GOVERNANCE      │                     │
│                    │  (Standards, Sécurité, ...)│                     │
│                    └─────────────────────────────┘                     │
│                                                                         │
└─────────────────────────────────────────────────────────────────────────┘
```

---

### 💡 Data Mesh vs Architecture Centralisée

| Aspect | Centralisée | Data Mesh |
|--------|-------------|------------|
| **Ownership** | Équipe data centrale | Équipes domaines |
| **Scalabilité** | Limitée par l'équipe data | Scale avec l'organisation |
| **Connaissance métier** | Faible | Forte (experts domaine) |
| **Time-to-market** | Lent | Rapide |
| **Complexité** | Technique | Organisationnelle |

> ⚠️ **Attention** : Data Mesh n'est **pas pour tout le monde**. Il convient aux **grandes organisations** avec de nombreux domaines métiers distincts.

> 🔮 **Pour approfondir** : Le Data Mesh est un concept avancé que tu exploreras au niveau expert du bootcamp !

---

# 🗄️ 12. NoSQL — Au-delà des bases relationnelles

Jusqu'ici, tu as travaillé avec des bases **relationnelles** (SQL). Mais le Big Data a fait émerger un nouveau paradigme : **NoSQL**.

---

## 🤔 Pourquoi NoSQL ?

### Les limites des bases relationnelles face au Big Data

```
┌─────────────────────────────────────────────────────────────────┐
│              PROBLÈMES DES BASES SQL CLASSIQUES                 │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ❌ Schéma rigide → Difficile de changer la structure           │
│  ❌ Scale-up only → Un seul serveur (coûteux)                   │
│  ❌ Jointures → Lentes sur des milliards de lignes              │
│  ❌ ACID strict → Latence élevée                                │
│  ❌ Données variées → JSON, graphes mal gérés                   │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

### La solution NoSQL

```
┌─────────────────────────────────────────────────────────────────┐
│                    AVANTAGES NoSQL                              │
├─────────────────────────────────────────────────────────────────┤
│                                                                 │
│  ✅ Schéma flexible → S'adapte aux données                      │
│  ✅ Scale-out natif → Ajout de serveurs facile                  │
│  ✅ Pas de jointures → Données dénormalisées, rapides           │
│  ✅ Haute disponibilité → Tolérance aux pannes                  │
│  ✅ Modèles variés → Document, clé-valeur, graphe...            │
│                                                                 │
└─────────────────────────────────────────────────────────────────┘
```

> 💡 **NoSQL** = "Not Only SQL" (pas seulement SQL), pas "No SQL" !

---

## 📊 Les 4 types de bases NoSQL

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                         TYPES DE BASES NoSQL                                │
├─────────────────────┬─────────────────────┬─────────────────────┬───────────┤
│     📄 DOCUMENT     │   🔑 CLÉ-VALEUR    │    📊 COLONNES      │  🔗 GRAPHE │
├─────────────────────┼─────────────────────┼─────────────────────┼───────────┤
│                     │                     │                     │           │
│  {                  │   key1 → value1     │  Row1: col1, col2   │   (A)──→(B)│
│    "nom": "Alice", │   key2 → value2     │  Row2: col1, col3   │    │       │
│    "age": 30        │   key3 → value3     │  Row3: col2, col4   │    ↓       │
│  }                  │                     │                     │   (C)     │
│                     │                     │                     │           │
├─────────────────────┼─────────────────────┼─────────────────────┼───────────┤
│  MongoDB            │  Redis              │  Cassandra          │ Neo4j     │
│  Couchbase          │  Memcached          │  HBase              │ Amazon    │
│  Firestore          │  DynamoDB           │  ScyllaDB           │ Neptune   │
└─────────────────────┴─────────────────────┴─────────────────────┴───────────┘
```

---

### 📄 1. Bases Document (MongoDB, Couchbase)

Stockent des **documents JSON/BSON** avec un schéma flexible.

```json
// Document MongoDB
{
  "_id": "user123",
  "nom": "Alice Dupont",
  "email": "alice@example.com",
  "adresses": [
    {"type": "domicile", "ville": "Paris"},
    {"type": "travail", "ville": "Lyon"}
  ]
}
```

| Avantages | Inconvénients |
|-----------|---------------|
| Schéma flexible | Pas de jointures natives |
| Documents imbriqués | Données dupliquées |
| Requêtes riches | Transactions limitées |

**Cas d'usage** : Catalogues produits, profils utilisateurs, CMS, logs

---

### 🔑 2. Bases Clé-Valeur (Redis, Memcached)

Stockage ultra-simple : une **clé** → une **valeur**.

```
┌─────────────────┬────────────────────────────┐
│      CLÉ        │          VALEUR            │
├─────────────────┼────────────────────────────┤
│  user:123       │  {"nom": "Alice", ...}    │
│  session:abc    │  {"user_id": 123, ...}    │
│  cache:page:42  │  "<html>...</html>"       │
└─────────────────┴────────────────────────────┘
```

**Cas d'usage** : Cache, sessions, compteurs, files d'attente

---

### 📊 3. Bases Colonnes (Cassandra, HBase)

Optimisées pour les **écritures massives** et les **requêtes analytiques**.

**Cas d'usage** : IoT, time-series, logs, analytics

---

### 🔗 4. Bases Graphe (Neo4j, Amazon Neptune)

Optimisées pour les **relations complexes** entre entités.

```
         ┌─────────┐
         │  Alice  │
         └────┬────┘
              │ KNOWS
              ▼
         ┌─────────┐       WORKS_AT       ┌─────────────┐
         │   Bob   │ ──────────────────►  │  TechCorp   │
         └─────────┘                      └─────────────┘
```

**Cas d'usage** : Réseaux sociaux, recommandations, fraude, connaissances

---

## 🔺 Le théorème CAP

Le **théorème CAP** (Eric Brewer, 2000) est fondamental pour comprendre les choix de design des bases distribuées.

```
                         C
                    Consistency
                    (Cohérence)
                        /\
                       /  \
                      /    \
                     /  CA  \
                    /________\
                   /\        /\
                  /  \      /  \
                 / CP \    / AP \
                /______\  /______\
               A                    P
          Availability          Partition
         (Disponibilité)        Tolerance
```

### 📖 Les 3 propriétés

| Propriété | Signification |
|-----------|---------------|
| **C**onsistency | Tous les nœuds voient les mêmes données |
| **A**vailability | Le système répond toujours |
| **P**artition Tolerance | Fonctionne malgré des pannes réseau |

### ⚠️ Le théorème dit :

> **En cas de partition réseau, tu dois choisir entre Cohérence et Disponibilité.**

Tu ne peux avoir que **2 sur 3** !

---

### 🗂️ Classification des bases selon CAP

| Type | Choix CAP | Bases | Comportement |
|------|-----------|-------|-------------|
| **CP** | Cohérence + Partition | MongoDB, HBase, Redis | Peut refuser des requêtes si partition |
| **AP** | Disponibilité + Partition | Cassandra, DynamoDB, CouchDB | Répond toujours, cohérence éventuelle |
| **CA** | Cohérence + Disponibilité | PostgreSQL, MySQL (single node) | Pas de tolérance aux partitions |

---

## ⚖️ SQL vs NoSQL — Quand utiliser quoi ?

### 📊 Tableau comparatif

| Critère | SQL (Relationnel) | NoSQL |
|---------|-------------------|-------|
| **Schéma** | Fixe, défini à l'avance | Flexible, dynamique |
| **Données** | Structurées, normalisées | Semi/non-structurées |
| **Relations** | Jointures natives | Données embarquées |
| **Transactions** | ACID complet | BASE (éventuel) |
| **Scalabilité** | Verticale (scale-up) | Horizontale (scale-out) |

---

### ✅ Utilise SQL quand...

- Données très structurées (comptabilité, RH)
- Relations complexes (ERP, CRM)
- Transactions critiques (banque, e-commerce)

### ✅ Utilise NoSQL quand...

- Schéma variable (catalogues, profils)
- Cache haute performance (sessions)
- Écritures massives (IoT, logs)
- Scale-out nécessaire (Big Data)

### 🎯 En réalité : on utilise les deux !

> 💡 **Polyglot Persistence** : Utiliser la bonne base pour le bon cas d'usage !

---

## 📋 Résumé

### Les 5V du Big Data

| V | Défi |
|---|------|
| **Volume** | Stocker et traiter des To/Po |
| **Velocity** | Traiter en temps réel |
| **Variety** | Gérer tous les formats |
| **Veracity** | Assurer la qualité |
| **Value** | Extraire des insights |

### Traitement distribué

| Concept | Retenir |
|---------|--------|
| **Scale-out** | Cluster de machines ordinaires |
| **Data locality** | Amener le code aux données |
| **Fault tolerance** | Réplication, recalcul |

### Hadoop vs Spark

| | Hadoop MR | Spark |
|---|-----------|-------|
| Stockage intermédiaire | Disque | Mémoire |
| Vitesse | Lent | 100x plus rapide |
| Langages | Java | Python, Scala, SQL |

### Architectures de stockage

| Type | Caractéristique clé |
|------|--------------------|
| **Data Lake** | Stockage brut, schema-on-read |
| **Data Lakehouse** | Lake + ACID + Performance |

### Architectures de traitement

| Architecture | Approche |
|--------------|----------|
| **Lambda** | Batch + Streaming (2 systèmes) |
| **Kappa** | Streaming-first (1 système) |

### SQL vs NoSQL

| Type | Quand utiliser |
|------|----------------|
| **SQL** | Données structurées, transactions, intégrité |
| **NoSQL Document** | Schéma flexible, JSON (MongoDB) |
| **NoSQL Clé-valeur** | Cache, sessions (Redis) |
| **NoSQL Colonnes** | Écritures massives, IoT (Cassandra) |
| **NoSQL Graphe** | Relations complexes (Neo4j) |

---

## 🧠 Quiz

---

### ❓ Q1. Quels sont les 3V originaux du Big Data ?
a) Vitesse, Valeur, Vérité  
b) Volume, Velocity, Variety  
c) Volume, Validation, Visualisation  
d) Vélocité, Véracité, Valorisation

<details><summary>💡 Réponse</summary>

✅ **b** — Les 3V originaux (Doug Laney, 2001) sont Volume, Velocity, Variety.
</details>

---

### ❓ Q2. Que signifie "Scale-out" ?
a) Augmenter la RAM d'un serveur  
b) Ajouter des machines au cluster  
c) Compresser les données  
d) Réduire la taille du cluster

<details><summary>💡 Réponse</summary>

✅ **b** — Scale-out (horizontal) = ajouter des machines. Scale-up (vertical) = augmenter les ressources d'une machine.
</details>

---

### ❓ Q3. Quel est le principe de "Data Locality" ?
a) Stocker les données localement sur son PC  
b) Déplacer les données vers le serveur de calcul  
c) Déplacer le code vers les données  
d) Compresser les données pour les transférer

<details><summary>💡 Réponse</summary>

✅ **c** — On envoie le code (petit) vers les données (grosses), pas l'inverse.
</details>

---

### ❓ Q4. Quels sont les 3 composants principaux de Hadoop ?
a) HDFS, Spark, Kafka  
b) HDFS, MapReduce, YARN  
c) Hive, Pig, HBase  
d) Map, Shuffle, Reduce

<details><summary>💡 Réponse</summary>

✅ **b** — HDFS (stockage), MapReduce (calcul), YARN (ressources).
</details>

---

### ❓ Q5. Pourquoi Spark est plus rapide que MapReduce ?
a) Il utilise un meilleur algorithme  
b) Il stocke les données intermédiaires en mémoire  
c) Il compresse les données  
d) Il utilise plus de serveurs

<details><summary>💡 Réponse</summary>

✅ **b** — Spark garde les données en mémoire (RAM) au lieu d'écrire sur disque entre chaque étape.
</details>

---

### ❓ Q6. Quelle est la différence entre Data Lake et Data Lakehouse ?
a) Aucune différence  
b) Le Lakehouse ajoute les transactions ACID au Data Lake  
c) Le Data Lake est plus récent  
d) Le Lakehouse ne supporte que les données structurées

<details><summary>💡 Réponse</summary>

✅ **b** — Le Data Lakehouse combine la flexibilité du Data Lake avec les transactions ACID et les performances du Data Warehouse.
</details>

---

### ❓ Q7. Quelles sont les 3 couches de l'architecture Lambda ?
a) Bronze, Silver, Gold  
b) Batch, Speed, Serving  
c) Extract, Transform, Load  
d) Input, Process, Output

<details><summary>💡 Réponse</summary>

✅ **b** — Lambda Architecture = Batch Layer + Speed Layer + Serving Layer.
</details>

---

### ❓ Q8. Quel est l'avantage principal de l'architecture Kappa sur Lambda ?
a) Plus rapide  
b) Plus simple (une seule codebase)  
c) Moins de stockage  
d) Meilleure précision

<details><summary>💡 Réponse</summary>

✅ **b** — Kappa simplifie l'architecture en n'utilisant qu'un seul système (streaming), évitant de maintenir deux codebases.
</details>

---

### ❓ Q9. Qu'est-ce qu'un Data Swamp ?
a) Un type de Data Lake optimisé  
b) Un Data Lake mal géré et inutilisable  
c) Une architecture de streaming  
d) Un Data Warehouse sur le cloud

<details><summary>💡 Réponse</summary>

✅ **b** — Un Data Swamp est un Data Lake devenu ingérable par manque de gouvernance (données non documentées, dupliquées, obsolètes).
</details>

---

### ❓ Q10. Quel principe du Data Mesh dit que chaque domaine métier est responsable de ses données ?
a) Data as a Product  
b) Self-serve Platform  
c) Domain Ownership  
d) Federated Governance

<details><summary>💡 Réponse</summary>

✅ **c** — Domain Ownership : chaque domaine métier possède et gère ses propres données.
</details>

---

### ❓ Q11. Combien de copies HDFS fait-il par défaut ?
a) 1  
b) 2  
c) 3  
d) 5

<details><summary>💡 Réponse</summary>

✅ **c** — HDFS réplique chaque bloc 3 fois par défaut pour la tolérance aux pannes.
</details>

---

### ❓ Q12. Quelle base NoSQL est idéale pour le cache et les sessions ?
a) MongoDB  
b) Redis  
c) Cassandra  
d) Neo4j

<details><summary>💡 Réponse</summary>

✅ **b** — Redis est une base in-memory clé-valeur, idéale pour le cache et les sessions.
</details>
---

### ❓ Q13. Dans l'architecture Medallion, quelle couche contient les données brutes non transformées ?
a) Gold  
b) Silver  
c) Bronze  
d) Platinum

<details><summary>💡 Réponse</summary>

✅ **c** — Bronze contient les données brutes exactement comme elles arrivent des sources.
</details>

---

### ❓ Q14. Quel est l'ordre correct de qualité croissante dans l'architecture Medallion ?
a) Gold → Silver → Bronze  
b) Bronze → Gold → Silver  
c) Silver → Bronze → Gold  
d) Bronze → Silver → Gold

<details><summary>💡 Réponse</summary>

✅ **d** — Bronze (brut) → Silver (nettoyé) → Gold (agrégé/business-ready). La qualité augmente à chaque couche.
</details>

---

## 📚 Ressources

### Big Data & Hadoop
- [Hadoop Documentation](https://hadoop.apache.org/docs/stable/)
- [Spark Documentation](https://spark.apache.org/docs/latest/)
- [The Google File System (paper)](https://research.google/pubs/pub51/)
- [MapReduce: Simplified Data Processing (paper)](https://research.google/pubs/pub62/)

### Architectures modernes
- [Delta Lake Documentation](https://docs.delta.io/)
- [Medallion Architecture (Databricks)](https://www.databricks.com/glossary/medallion-architecture)
- [Apache Iceberg](https://iceberg.apache.org/)
- [Lambda Architecture (Nathan Marz)](http://nathanmarz.com/blog/how-to-beat-the-cap-theorem.html)
- [Kappa Architecture (Jay Kreps)](https://www.oreilly.com/radar/questioning-the-lambda-architecture/)
- [Data Mesh (Zhamak Dehghani)](https://martinfowler.com/articles/data-mesh-principles.html)

### NoSQL
- [MongoDB University](https://university.mongodb.com/) — Cours gratuits
- [Redis Documentation](https://redis.io/docs/)
- [CAP Theorem Explained](https://www.ibm.com/topics/cap-theorem)

---

## ➡️ Prochaine étape

Tu as maintenant les bases théoriques du Big Data, du traitement distribué, des architectures modernes et du NoSQL. Place à la pratique avec MongoDB !

👉 **Module suivant : `09_mongodb_for_data_engineers.ipynb`** — MongoDB pour les Data Engineers

Tu apprendras à :
- Installer et utiliser MongoDB
- Créer des collections et documents
- Écrire des requêtes MQL (MongoDB Query Language)
- Utiliser PyMongo depuis Python
- Modéliser des données pour MongoDB

---

🎉 **Félicitations !** Tu comprends maintenant le Big Data, les systèmes distribués, les architectures modernes et le NoSQL.